<a href="https://colab.research.google.com/github/ErikaNat/Inteligencia-Artificial/blob/main/Ejercicio10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

Open In Colab

import os
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras import layers, models, callbacks


SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TensorFlow:", tf.__version__)

TensorFlow: 2.19.0

df = pd.read_csv("student_exam_scores.csv")
print("Shape:", df.shape)
print("Cols:", list(df.columns))

Shape: (200, 6)
Cols: ['student_id', 'hours_studied', 'sleep_hours', 'attendance_percent', 'previous_scores', 'exam_score']

X = df[['hours_studied', 'sleep_hours', 'attendance_percent', 'previous_scores']].copy()
y = df['exam_score'].values


num_cols = ['hours_studied', 'sleep_hours', 'attendance_percent', 'previous_scores']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols)
    ],
    remainder='drop'
)


X_train_df, X_test_df, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

X_train = preprocessor.fit_transform(X_train_df)
X_test = preprocessor.transform(X_test_df)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print("Input dims:", X_train.shape[1])

Input dims: 4

def build_model(input_dim: int) -> tf.keras.Model:
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(1)
    ])
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    return model

model = build_model(X_train.shape[1])
model.summary()

Model: "sequential"


cbs = [
    callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6),
    callbacks.ModelCheckpoint('students_best.keras', monitor='val_loss', save_best_only=True)
]


hist = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=16,
    callbacks=cbs,
    verbose=1
)



y_pred = model.predict(X_test).ravel()

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nMAE:", mae)
print("RMSE:", np.sqrt(mse))
print("R²:", r2)


